In [1]:
import pandas as pd
from gp_list_design_tree import *

Проверяем базовый класс от которого потом будут наследоваться остальные узлы. 
Проверяем, что класс создается, параметры устанавливаются

In [2]:
l=list_tree_base()
print(f'get_name: {l.get_name()}')
l.name='test'
l.num_childs=2
l.num_koef=3
l.koef={'p1':-1, 'p2':-2}
print(f'get_name: {l.get_name()}')

get_name: {}
get_name: test{'p1': -1, 'p2': -2}


Проверяем метод копирования узла в базовом классе.

In [3]:
l1=l.copy()
print(f'get_name: {l1.get_name()}')
print(l1.get_name_koef())
print(l1.get_koef())

get_name: test{'p1': -1, 'p2': -2}
[]
{'p1': -1, 'p2': -2}


Проверяем работу терминального узла для классификации, который строится наследованием от базового класса
Создаем маску, Устанавливаем номер класса равный единице. Ожидаем, что для всех значений удовлетворяющих маске вернется этоот номер класса

In [4]:
y=pd.Series(np.arange(0,1, 0.1), index=100*np.arange(0,1, 0.1))
node=list_nom_class(value=1)
# koef={'class':1}
# node.set_koef(koef)
rez=node.eval(params={'y':y}, mask=y>0.5)
node1=node.copy()
print(node.get_name())
print(rez)

nom_class:1
0.0     NaN
10.0    NaN
20.0    NaN
30.0    NaN
40.0    NaN
50.0    NaN
60.0    1.0
70.0    1.0
80.0    1.0
90.0    1.0
dtype: float64


Проверяем регрессионный узел. Устанавливаем маску и значение константы узла равное 101. Ожидаем, что для значений удовлетворяющих маске установится это значение регрессионного узла

In [5]:
node=list_regr_const()
koef={'const':101}
node.set_koef(koef)
rez=node.eval(params={'y':y, }, mask=y<0.5)
node1=node.copy()
print(node.get_name())
print(rez)

const{'const': 101}
0.0     101.0
10.0    101.0
20.0    101.0
30.0    101.0
40.0    101.0
50.0      NaN
60.0      NaN
70.0      NaN
80.0      NaN
90.0      NaN
dtype: float64


Проверяем работу функционального узал по типу x<p, где p это параметр. Ожидается, что вернется маска, где True для удовлетворяющих условию и False для неудовлетворяющих условию

In [6]:
df=y.copy()
df=df*1000
df.name='x1'
df=df.reset_index()
node=list_less(name_feature='x1' )
node1=node.copy()
koef={'p':101}
node.set_koef(koef)
rez=node.eval(params={'X':df,'y':y}  )
print(node.get_name())
print(rez)
print(df)


less[x1,{'p': 101}]
0     True
1     True
2    False
3    False
4    False
5    False
6    False
7    False
8    False
9    False
Name: x1, dtype: bool
   index     x1
0    0.0    0.0
1   10.0  100.0
2   20.0  200.0
3   30.0  300.0
4   40.0  400.0
5   50.0  500.0
6   60.0  600.0
7   70.0  700.0
8   80.0  800.0
9   90.0  900.0


проверяем в узле list_less самооптимизацию с фиксированными классами на метрике gini

In [7]:
import pandas as pd
from gp_list_design_tree import *
df=pd.Series(np.arange(1,0, -0.1), index=100*np.arange(0,1, 0.1))

df.name='x1'
df=df.reset_index()
y=pd.Series(np.zeros(len(df), dtype=int))
y.loc[df['x1']>0.5]=1

node=list_less(name_feature='x1' )
node1=node.copy()
mask=np.array([True]*len(df))

rez=node.optim_koef(params={'X':df,'y':y}, mask0=mask)

print(rez)
print(df)

{'score': False, 'inf_gate': 0.48, 'mask0': 0    False
1    False
2    False
3    False
4    False
5    False
6     True
7     True
8     True
9     True
Name: x1, dtype: bool, 'mask1': 0     True
1     True
2     True
3     True
4     True
5     True
6    False
7    False
8    False
9    False
Name: x1, dtype: bool}
   index   x1
0    0.0  1.0
1   10.0  0.9
2   20.0  0.8
3   30.0  0.7
4   40.0  0.6
5   50.0  0.5
6   60.0  0.4
7   70.0  0.3
8   80.0  0.2
9   90.0  0.1


Проверяем самооптимизацию в узле list_less со случайными классами на метрике gini

In [8]:
import pandas as pd
from gp_list_design_tree import *
df=pd.Series(np.arange(1,0, -0.1), index=100*np.arange(0,1, 0.1))

df.name='x1'
df=df.reset_index()
y=pd.Series(np.random.randint(0, 2, len(df), dtype=int))
# y.loc[df['x1']>0.5]=1

node=list_less(name_feature='x1' )
node1=node.copy()
mask=np.array([True]*len(df))

rez=node.optim_koef(params={'X':df,'y':y}, mask0=mask)

print(rez)
print(df)

{'score': False, 'inf_gate': 0.125, 'mask0': 0    False
1    False
2    False
3    False
4    False
5    False
6    False
7    False
8     True
9     True
Name: x1, dtype: bool, 'mask1': 0     True
1     True
2     True
3     True
4     True
5     True
6     True
7     True
8    False
9    False
Name: x1, dtype: bool}
   index   x1
0    0.0  1.0
1   10.0  0.9
2   20.0  0.8
3   30.0  0.7
4   40.0  0.6
5   50.0  0.5
6   60.0  0.4
7   70.0  0.3
8   80.0  0.2
9   90.0  0.1


проверяем работу самооптимизации в узле list_less и реакцию на перемещение порогов на метрике gini

In [9]:
import pandas as pd
from gp_list_design_tree import *
df=pd.Series(np.arange(1,0, -0.1), index=100*np.arange(0,1, 0.1))

df.name='x1'
df=df.reset_index()
y=pd.Series(np.zeros(len(df), dtype=int))
y.loc[df['x1']>0.6]=1

node=list_less(name_feature='x1' )
node1=node.copy()
mask=np.array([True]*len(df))
# koef={'p':101}
# node.set_koef(koef)
# rez=node.eval(params={'X':df,'y':y}  )
rez=node.optim_koef(params={'X':df,'y':y}, mask0=mask, inf_name='gini' )
df['y']=y
print(rez)
print(df)

{'score': False, 'inf_gate': 0.5, 'mask0': 0    False
1    False
2    False
3    False
4    False
5     True
6     True
7     True
8     True
9     True
Name: x1, dtype: bool, 'mask1': 0     True
1     True
2     True
3     True
4     True
5    False
6    False
7    False
8    False
9    False
Name: x1, dtype: bool}
   index   x1  y
0    0.0  1.0  1
1   10.0  0.9  1
2   20.0  0.8  1
3   30.0  0.7  1
4   40.0  0.6  1
5   50.0  0.5  0
6   60.0  0.4  0
7   70.0  0.3  0
8   80.0  0.2  0
9   90.0  0.1  0


проверяем работу самооптимизации в узле list_less  на метрике mse

In [10]:
import pandas as pd
from gp_list_design_tree import *
df=pd.Series(np.arange(1,0, -0.1), index=100*np.arange(0,1, 0.1))

df.name='x1'
df=df.reset_index()
y=pd.Series(np.linspace(0,1, len(df)))
y=y**2

node=list_less(name_feature='x1' )
node1=node.copy()
mask=np.array([True]*len(df))

rez=node.optim_koef(params={'X':df,'y':y}, mask0=mask, inf_name='mse' )
df['y']=y
print(rez)
print(df)
print(node.koef)

{'score': False, 'inf_gate': 0.09097190468932576, 'mask0': 0    False
1    False
2    False
3    False
4    False
5    False
6    False
7     True
8     True
9     True
Name: x1, dtype: bool, 'mask1': 0     True
1     True
2     True
3     True
4     True
5     True
6     True
7    False
8    False
9    False
Name: x1, dtype: bool}
   index   x1         y
0    0.0  1.0  0.000000
1   10.0  0.9  0.012346
2   20.0  0.8  0.049383
3   30.0  0.7  0.111111
4   40.0  0.6  0.197531
5   50.0  0.5  0.308642
6   60.0  0.4  0.444444
7   70.0  0.3  0.604938
8   80.0  0.2  0.790123
9   90.0  0.1  1.000000
{'p': 0.35000000000000014}


проверяем работу самооптимизации в узле list_regr_const  на метрике mse

In [11]:
import pandas as pd
from gp_list_design_tree import *
df=pd.Series(np.arange(1,0, -0.1), index=100*np.arange(0,1, 0.1))

df.name='x1'
df=df.reset_index()
y=pd.Series(np.linspace(0,1, len(df)))
y=y**0.5

node=list_regr_const( )

mask=np.array([True]*len(df))

rez=node.optim_koef(params={'X':df,'y':y}, mask0=mask)
df['y']=y
print(rez)
print(df)
print(node.koef)

{'score': 0.0, 'inf_gate': False}
   index   x1         y
0    0.0  1.0  0.000000
1   10.0  0.9  0.333333
2   20.0  0.8  0.471405
3   30.0  0.7  0.577350
4   40.0  0.6  0.666667
5   50.0  0.5  0.745356
6   60.0  0.4  0.816497
7   70.0  0.3  0.881917
8   80.0  0.2  0.942809
9   90.0  0.1  1.000000
{'const': 0.6435333508678573}


проверяем работу самооптимизации в узле list_nom_class  на метрике mse

In [12]:
import pandas as pd
from gp_list_design_tree import *
df=pd.Series(np.arange(1,0, -0.1), index=100*np.arange(0,1, 0.1))

df.name='x1'
df=df.reset_index()
y=pd.Series(np.random.randint(0, 2, len(df), dtype=int))


node=list_nom_class( )

mask=np.array([True]*len(df))

rez=node.optim_koef(params={'y':y}, mask0=mask)
df['y']=y
print(rez)
print(df)
print(node.value)

{'score': 0.37499999999999994, 'inf_gate': False}
   index   x1  y
0    0.0  1.0  1
1   10.0  0.9  0
2   20.0  0.8  0
3   30.0  0.7  0
4   40.0  0.6  0
5   50.0  0.5  1
6   60.0  0.4  0
7   70.0  0.3  0
8   80.0  0.2  1
9   90.0  0.1  1
0


In [20]:
l=['A', 'B', 1, 2,30]
l=pd.Series(l)
l1=l.sample(2)
list(l1)

[2, 'B']